# Introduction
In this notebook, we'll focus on data wrangling. Let's outline the steps and the goals of each step.

|Step               |Goal                                                                   |
|-------------------|-----------------------------------------------------------------------|
|Data Collection    |Gather and join the data to streamline the next steps of the capstone  |
|Data Organization: |Establish the file structure and version control                       |
|Data Definition:   |Understand, annotate and clean the data in preparation for future work |
|Data Cleaning:     |Check for missing data or wrong data, and handle them appropriately    |

# Data Collection

## Data Loading

We'll import the data from Kaggle. 

Kaggle requires users to sign in and generate an API Key. Make sure your API key is at the correct location before running the next cell. If necessary, also make sure that Kaggle has been installed before continuing.


In [1]:
# Download the zipped data using the Kaggle API 
!kaggle competitions download -c riiid-test-answer-prediction -p "..\data\raw"

riiid-test-answer-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
#Unzip the downloaded file
from zipfile import ZipFile
from pathlib import Path

zipped_data_path = Path('../data/raw/riiid-test-answer-prediction.zip')
unzip_destination_folder_path = Path('../data/interim')

with ZipFile(zipped_data_path, 'r') as zf:
    # Save list of file names in zip file to a list
    zf_names = zf.namelist()
    # Extract all files
    zf.extractall(unzip_destination_folder_path)

In [3]:
#Check the names of the unzipped files for the file names containing our data.
zf_names

['example_sample_submission.csv',
 'example_test.csv',
 'lectures.csv',
 'questions.csv',
 'riiideducation/__init__.py',
 'riiideducation/competition.cpython-37m-x86_64-linux-gnu.so',
 'train.csv']

The data_csv_files are  `lectures.csv`,  `questions.csv`, `train.csv` .

In [4]:
import pandas as pd

#Define data paths
lectures_csv_path = Path('../data/interim/lectures.csv')
questions_csv_path = Path('../data/interim/questions.csv')
train_csv_path = Path('../data/interim/train.csv')

# For these small csv files, import them directly with pandas.read_csv()
df_lectures = pd.read_csv(lectures_csv_path)
df_questions = pd.read_csv(questions_csv_path)

`train.csv` is a large csv file, over 7 GB and 100M rows, so we need to load it in chunks.

In [8]:
# Make DataFrame generator from CSV in chunks
df_generator = pd.read_csv(train_csv_path, chunksize=10000000)

#Initialize an empty DataFrame: df_train
df_train = pd.DataFrame()

# Iterate over each DataFrame chunk
for df_chunk in df_generator:
    df_train = df_train.append(df_chunk)


After importing as a dataframe, save the dataframe as a binary file, so that we can quickly reload the dataframe and resume.


In [9]:
# Define paths
lectures_pkl_path = Path('../data/interim/lectures.pkl.gzip')
questions_pkl_path = Path('../data/interim/questions.pkl.gzip')
train_pkl_path = Path('../data/interim/train.pkl.gzip')

# Save DataFrames to as pkl
df_lectures.to_pickle(lectures_pkl_path)
df_questions.to_pickle(questions_pkl_path)
df_train.to_pickle(train_pkl_path)

Check the heads of the three DataFrames

In [30]:
df_lectures.head()

,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [31]:
df_questions.head()

,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [32]:
df_train.head()

,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,NaN
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


## Data Joining

The data is available as one large file. So their is no need to join the data other than joining the chunks of the large file while importing it.

## Data Subsetting with the larger dataframes
As the data is quite large, it might be useful to subset the data during exploratory data analysis to speed up the process.

In [10]:
#Define the row skip logic

#Skip rows from based on condition like skip every 10th line
def skip_all_but_nth_rows(n, idx):
  return (idx % n != 0)
  
#Skip random lines  
import random
def rand_1_in_n(n, idx):
  return True if random.randrange(1,n)==1 else False


#Create the subsets 

#Define a Dataframe with 1/10 of the data
df_train_1_10 = df_train[df_train.index % 10 == 0]

#Define a DataFrame with 1/100 of the data
df_train_1_100 = df_train[df_train.index % 100 == 0]

#Define a DataFrame with 1/1000 of the data
df_train_1_1000 = df_train[df_train.index % 1000 == 0]

#Define a Dataframe with 1/10000 of the data
df_train_1_10000 = df_train[df_train.index % 10000 == 0]

#Define a Dataframe with 1/100000 of the data
df_train_1_100000 = df_train[df_train.index % 100000 == 0]

#Define a Dataframe with 1/1000000 of the data
df_train_1_1000000 = df_train[df_train.index % 1000000 == 0]

#Define a Dataframe with 1/10000000 of the data
df_train_1_10000000 = df_train[df_train.index % 10000000 == 0]


#Define subset paths
train_pkl_path_1_10 = Path('../data/interim/train_1_10.pkl.gzip')
train_pkl_path_1_100 = Path('../data/interim/train_1_100.pkl.gzip')
train_pkl_path_1_1000 = Path('../data/interim/train_1_1000.pkl.gzip')
train_pkl_path_1_10000 = Path('../data/interim/train_1_10000.pkl.gzip')
train_pkl_path_1_100000 = Path('../data/interim/train_1_100000.pkl.gzip')
train_pkl_path_1_1000000 = Path('../data/interim/train_1_1000000.pkl.gzip')
train_pkl_path_1_10000000 = Path('../data/interim/train_1_10000000.pkl.gzip')

#Save subset dataframes to pkl
df_train_1_10.to_pickle(train_pkl_path_1_10)
df_train_1_100.to_pickle(train_pkl_path_1_100)
df_train_1_1000.to_pickle(train_pkl_path_1_1000)
df_train_1_10000.to_pickle(train_pkl_path_1_10000)
df_train_1_100000.to_pickle(train_pkl_path_1_100000)
df_train_1_1000000.to_pickle(train_pkl_path_1_1000000)
df_train_1_10000000.to_pickle(train_pkl_path_1_10000000)

## Resuming data analysis without reimporting data from source files
After the binary files have been saved, we can quickly resume by loading the binary files rather tha downloading, unzipping, and reading the csv files in chunks, again.

In [11]:
import pickle
import pandas as pd

# Define paths
lectures_pkl_path = Path('../data/interim/lectures.pkl.gzip')
questions_pkl_path = Path('../data/interim/questions.pkl.gzip')
train_pkl_path = Path('../data/interim/train.pkl.gzip')

with open(lectures_pkl_path, 'rb') as f:
    df_lectures = pickle.load(f)
    
with open(questions_pkl_path, 'rb') as f:
    df_questions = pickle.load(f)
    
with open(train_pkl_path, 'rb') as f:
    df_train = pickle.load(f)
    
# Check the shape of the dataframes
df_lectures.shape, df_questions.shape, df_train.shape

((418, 4), (13523, 5), (101230332, 10))

### Loading the subsets from the pkl files


In [12]:
#Define subset paths
train_pkl_path_1_10 = Path('../data/interim/train_1_10.pkl.gzip')
train_pkl_path_1_100 = Path('../data/interim/train_1_100.pkl.gzip')
train_pkl_path_1_1000 = Path('../data/interim/train_1_1000.pkl.gzip')
train_pkl_path_1_10000 = Path('../data/interim/train_1_10000.pkl.gzip')
train_pkl_path_1_100000 = Path('../data/interim/train_1_100000.pkl.gzip')
train_pkl_path_1_1000000 = Path('../data/interim/train_1_1000000.pkl.gzip')
train_pkl_path_1_10000000 = Path('../data/interim/train_1_10000000.pkl.gzip')

#Load the subsets
with open( train_pkl_path_1_10, 'rb') as f:
  df_train_1_10 = pickle.load(f)
with open( train_pkl_path_1_100, 'rb') as f:
  df_train_1_100 = pickle.load(f)
with open( train_pkl_path_1_1000, 'rb') as f:
  df_train_1_1000 = pickle.load(f)
with open( train_pkl_path_1_10000, 'rb') as f:
  df_train_1_10000 = pickle.load(f)
with open( train_pkl_path_1_100000, 'rb') as f:
  df_train_1_100000 = pickle.load(f)
with open( train_pkl_path_1_1000000, 'rb') as f:
  df_train_1_1000000 = pickle.load(f)
with open( train_pkl_path_1_10000000, 'rb') as f:
  df_train_1_10000000 = pickle.load(f)

# Store a list of the subset DataFrames
df_train_subsets = [df_train_1_10, 
                    df_train_1_100, 
                    df_train_1_1000,
                    df_train_1_10000,
                    df_train_1_100000,
                    df_train_1_1000000,
                    df_train_1_10000000,]

# Check the shape of the subset dataframes
df_train_1_10.shape, df_train_1_100.shape, df_train_1_1000.shape

((10123034, 10), (1012304, 10), (101231, 10))

# Data Organization

## File Structure

We'll use the default [file structure template for data science from cookiecutter data science](https://medium.com/@rrfd/cookiecutter-data-science-organize-your-projects-atom-and-jupyter-2be7862f487e).

The data files have already been imported according to this template.




|             |                  |
|-------------|------------------|
|├── README.md|          <- Front page of the project. Let everyone |
|│|                         know the major points.|
|│|
|├── models|             <- Trained and serialized models, model|
|│|                         predictions, or model summaries.|
|│|
|├── notebooks|          <- Jupyter notebooks. Use set naming|
|│|                         E.g. `1.2-rd-data-exploration`.|
|│|
|├── reports|            <- HTML, PDF, and LaTeX.|
|│   └── figures|        <- Generated figures.|
|│|
|├── requirements.txt|   <- File for reproducing the environment|
|│|                         `$ pip freeze > requirements.txt`|
|├── data|
|│   ├── external|       <- Third party sources.|
|│   ├── interim|        <- In-progress intermediate data.|
|│   ├── processed|      <- The final data sets for modelling.|
|│   └── raw|            <- The original, immutable data.|
|│|
|└── src |               <- Source code for use in this project.|
|    ├── __init__.py|    <- Makes src a Python module. |
|    │|
|    ├── custom_func.py| <- Various custom functions to import.|
|    │|
|    ├── data          | <- Scripts to download or generate data.|
|    │   └── make_dataset.py|
|    │|
|    ├── features|       <- Scripts raw data into features for|
|    │   │        |         modeling.|
|    │   └── build_features.py|
|    │|
|    ├── models|         <- Scripts to train models and then use|
|    │   │     |            trained models to make predictions.|
|    │   │     |            
|    │   ├── predict_model.py|
|    │   └── train_model.py|
|    │|
|    └── viz|            <- Scripts to create visualizations.|            
|        └── viz.py|

## Version Control

This notebook and it's related files will be stored in a local repository and on Github at:
https://github.com/allen44/capstone-2

## Environmental variables
Following the best practices outlined in the [Twelve Factor App](https://12factor.net/), environmental variables will be excluded from version control.

FOr this notebook, that means that any user wishing to reproduce the data loading steps will need their own Kaggle API key.

# Data Definition

Kaggle lists the definitions of the data on the [competition webpage](https://www.kaggle.com/c/riiid-test-answer-prediction/data).

Here's a excerpt of the relevant section:


### lectures.csv: metadata for the lectures watched by users as they progress in their education.
>`lecture_id`: foreign key for the train/test content_id column, when the content type is lecture (1).

>`part`: top level category code for the lecture.

>`tag`: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

>`type_of`: brief description of the core purpose of the lecture

### questions.csv: metadata for the questions posed to users.
>`question_id`: foreign key for the train/test content_id column, when the content type is question (0).

>`bundle_id`: code for which questions are served together.

>`correct_answer`: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

>`part`: the relevant section of the TOEIC test.

>`tags`: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

### train.csv 
>`content_id`: (int16) ID code for the user interaction

>`content_type_id`: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

>`task_container_id`: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

>`user_answer`: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

>`answered_correctly`: (int8) if the user responded correctly. Read -1 as null, for lectures.

>`prior_question_elapsed_time`: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

>`prior_question_had_explanation`: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.


## Set the dtypes

Based on the Kaggle eplanatoins of the data columns, we have enough info to set the dtypes on the DataFrames, and label missing or null data.

In [13]:
#Check current dtypes
print('df_lectures\n', df_lectures.dtypes, '\n')
print('df_questions\n', df_questions.dtypes,  '\n')
print('df_train\n', df_train.dtypes)

df_lectures
 lecture_id     int64
tag            int64
part           int64
type_of       object
dtype: object 

df_questions
 question_id        int64
bundle_id          int64
correct_answer     int64
part               int64
tags              object
dtype: object 

df_train
 row_id                              int64
timestamp                           int64
user_id                             int64
content_id                          int64
content_type_id                     int64
task_container_id                   int64
user_answer                         int64
answered_correctly                  int64
prior_question_elapsed_time       float64
prior_question_had_explanation     object
dtype: object


We can see that the dtypes don't match the data as described by the Kaggle data description.

In [14]:
# Define new dtypes
lectures_dtypes = {'lecture_id': 'category',
                    'part': 'category',
                    'tag': 'category',
                    'type_of': 'string'}
            
questions_dtypes = {'question_id': 'category', 
                   'bundle_id': 'category',
                   'correct_answer': 'category', 
                   'part': 'category',
                   'tags': 'string'}
            
train_dtypes = {'row_id': 'category',
                'timestamp': 'int64',
                'user_id': 'category',
                'content_id': 'category', 
                'content_type_id': 'category',
                'task_container_id': 'category', 
                'user_answer': 'category',
                'answered_correctly': 'category',
                'prior_question_elapsed_time': 'float',
                'prior_question_had_explanation': 'category'}


In [15]:
# Set the dtypes
df_lectures = df_lectures.astype(lectures_dtypes)
df_questions = df_questions.astype(questions_dtypes)
df_train = df_train.astype(train_dtypes)

#Check the new dtypes
print('df_lectures\n', df_lectures.dtypes, '\n')
print('df_questions\n', df_questions.dtypes,  '\n')
print('df_train\n', df_train.dtypes)

df_lectures
 lecture_id    category
tag           category
part          category
type_of         string
dtype: object 

df_questions
 question_id       category
bundle_id         category
correct_answer    category
part              category
tags                string
dtype: object 

df_train
 row_id                            category
timestamp                            int64
user_id                           category
content_id                        category
content_type_id                   category
task_container_id                 category
user_answer                       category
answered_correctly                category
prior_question_elapsed_time        float64
prior_question_had_explanation    category
dtype: object


In [16]:
#Also, set the dtypes on train subset DataFrames
df_train_1_10 = df_train_1_10.astype(train_dtypes)
df_train_1_100 = df_train_1_100.astype(train_dtypes)
df_train_1_1000 = df_train_1_1000.astype(train_dtypes)
df_train_1_10000 = df_train_1_10000.astype(train_dtypes)
df_train_1_100000 = df_train_1_100000.astype(train_dtypes)
df_train_1_1000000 = df_train_1_1000000.astype(train_dtypes)
df_train_1_10000000 = df_train_1_10000000.astype(train_dtypes)

#Check dtypes
print('df_train_1_10: \n', df_train_1_10.dtypes)
print('df_train_1_1000: \n', df_train_1_1000.dtypes)

df_train_1_10: 
 row_id                            category
timestamp                            int64
user_id                           category
content_id                        category
content_type_id                   category
task_container_id                 category
user_answer                       category
answered_correctly                category
prior_question_elapsed_time        float64
prior_question_had_explanation    category
dtype: object
df_train_1_1000: 
 row_id                            category
timestamp                            int64
user_id                           category
content_id                        category
content_type_id                   category
task_container_id                 category
user_answer                       category
answered_correctly                category
prior_question_elapsed_time        float64
prior_question_had_explanation    category
dtype: object


## Set Dummy Variables

`tags` is a compound variable.

The entries in the `tags` column in `df_questions` are numerical ids seperated by spaces where each id is a category that corresponds to the subject matter of the question. Many questions have more than one id in `tags`, though some questions only have one. 

We need to use `Dataframe.get_dummy()` to seperate the compound variable.

In [ ]:
#Check for missing values in tags

### Pandas Profiling Report
The Pandas Profiling module is a quick way to get an overview of the data sets. For the largest dataset, we will make a profile report on the subset only.

In [17]:
from pandas_profiling import ProfileReport
from pathlib import Path

#Define Pandas Profile Report save path
pandas_profiling_report_dir = Path('../reports')

#Generate Pandas Profiling Report - may be slow running if using the largest subsets of the data

report_lectures= ProfileReport(df_lectures, sort='None', html={'style':{'full_width': True}}, progress_bar=False)
report_questions = ProfileReport(df_questions, sort='None', html={'style':{'full_width': True}}, progress_bar=False)
report_train_1_100000= ProfileReport(df_train_1_100000, sort='None', html={'style':{'full_width': True}}, progress_bar=False)

#Save reports to file
report_lectures.to_file(pandas_profiling_report_dir / 'lectures.html')
report_questions.to_file(pandas_profiling_report_dir / 'questions.html')
report_train_1_100000.to_file(pandas_profiling_report_dir / 'train_1_100000.html')

In the Pandas Profiling reports, we can see that the data is pretty clean, as expected for a Kaggle dataset.

# Data Cleaning


## Check for missing data

In [18]:
df_lectures.isna().sum()

lecture_id    0
tag           0
part          0
type_of       0
dtype: int64

In [19]:
df_questions.isna().sum()

question_id       0
bundle_id         0
correct_answer    0
part              0
tags              1
dtype: int64

In [20]:
df_train.isna().sum()

row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
dtype: int64

We can see that `lectures` has no missing data, `questions` has one missing tag, and `train` has 2 million (about 2%) missing `prior_question_elapsed_time` entries and less than 1% missing entries in `prior_question_had_explanation`. Based on the explanation of the data from Kaggle, we can conclude that this missing data is normal. Nothing should be discard or imputed at this stage.

Note that the target variable, `answered_correctly`, has no missing data. This is good.

## Check for duplicated data

In [24]:
print("Fraction of duplicates in df_lectures:")
df_lectures.duplicated().sum() / len(df_lectures)

Fraction of duplicates in df_lectures:


0.0

In [25]:
print("Fraction of duplicates in df_questions:")
df_questions.duplicated().sum() / len(df_questions)

Fraction of duplicates in df_lectures:


0.0

In [29]:
print("Fraction of duplicates in df_train (df_train_1_100):")
df_train_1_100.duplicated().sum() / len(df_train_1_100)

Fraction of duplicates in df_train (df_train_1_100):


0.0

Again, as this Kaggle competition dataset, which are known to be fairly clean, there is no duplicated entries.